In [ ]:
import d2l_helper
import random
import matplotlib.pyplot as plt

In [ ]:
#人造数据集
def synthetic_data(w,b,num_examples):
    X=d2l_helper.normal(0,1,(num_examples,len(w)))
    y=d2l_helper.matmul(X,w)+b
    y+=d2l_helper.normal(0,0.01,y.shape)
    return X,y.reshape(-1,1)

In [ ]:
def data_iter(batch_size,features,labels):
    num_examples=len(features)
    indices=list(range(num_examples))
    random.shuffle(indices)
    for i in range(0,num_examples,batch_size):
        batch_indices=d2l_helper.tensor(indices[i:min(i+batch_size,num_examples)])
    yield features[batch_indices],labels[batch_indices]


In [4]:
#have a little try of data_iter output,生成第一轮i=0的batch就break
true_batch_size=10
#注意这个地方的true_w必须要是张量类型
true_w=d2l_helper.tensor([2.5,3.7])
true_b=0.01
features_1,labels_1=synthetic_data(true_w,true_b,1000)
for X,y in data_iter(true_batch_size,features_1,labels_1):
    print(X,'\n',y)
    break

tensor([[-0.2038, -0.2301],
        [ 0.6003,  0.8560],
        [ 0.7269,  0.2439],
        [-0.9086, -0.4832],
        [-0.0400, -0.7127],
        [ 0.7884,  0.0021],
        [ 0.3913,  0.5942],
        [-0.6643,  1.4171],
        [ 0.0574, -0.1358],
        [ 1.7468, -2.0483]]) 
 tensor([[-1.3495],
        [ 4.6641],
        [ 2.7439],
        [-4.0493],
        [-2.7416],
        [ 1.9783],
        [ 3.1867],
        [ 3.6053],
        [-0.3524],
        [-3.1930]])


In [ ]:
#define the model
def linreg(X,w,b):
    return d2l_helper.matmul(X,w)+b

In [6]:
def squared_loss(y_hat,y):
    y=y.reshape(-1,1)
    return (y_hat-y)**2/2

In [ ]:
#define the optimization method
def sgd(params,lr,batch_size):
#临时禁用梯度计算，torch.no_grad()
    with d2l_helper.no_grad():
        for param in params:
#自动微分机制，w定义的时候用了requires_grad=True
            param-=lr*param.grad/batch_size
            param.grad.zero_()


In [8]:
#begin training
lr=0.03
num_epochs=3
net=linreg
loss=squared_loss

In [ ]:
w=d2l_helper.normal(0,1,(2,1),requires_grad=True)
b=d2l_helper.zeros(1,requires_grad=True)

In [10]:
print("X shape:", features_1.shape)
print("y shape:", labels_1.shape)
print("y_hat shape:", net(features_1, w, b).shape)


X shape: torch.Size([1000, 2])
y shape: torch.Size([1000, 1])
y_hat shape: torch.Size([1000, 1])


In [11]:
num_epochs=3
for epoch in range(num_epochs):
    for X,y in data_iter(true_batch_size,features_1,labels_1):
    #l是当前批次损失，但是train_l是全量训练集损失
        l=loss(net(X,w,b),y)
        l.sum().backward(retain_graph=True)
        sgd([w,b],lr,true_batch_size)
    with d2l_helper.no_grad():
        train_l=loss(net(features_1,w,b),labels_1)
        print(f'epoch{epoch+1},loss{float(train_l.mean()):f}')


epoch1,loss4.391857
epoch2,loss3.960859
epoch3,loss3.647053
